In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

import numpy as np
import os
import pandas as pd
from IPython.display import Image, display, HTML, Markdown, FileLink
from pathlib import Path

# Helper functions

## For Quantitative Results

In [ ]:
def get_experiment_df(exp_path, multirun=False, exclude=None):
    variant_dfs = []
    for variant_dir in os.listdir(exp_path):
        if os.path.isfile(exp_path / variant_dir):
            continue

        # get runs inside each variant_dir 
        run_dirs = [run_dir for run_dir in os.listdir(exp_path / variant_dir)
                if '.yaml' not in run_dir]
        if len(run_dirs) == 0:
            continue

        # get metrics from each run
        run_dfs = []
        if multirun:
            for run_dir in run_dirs:
                run_df = pd.read_csv(exp_path / variant_dir / run_dir / 'metrics.csv')
                if exclude is not None:
                    run_df = run_df[~run_df['object_name'].isin([exclude])]
                # Add run dir as another column
                run_df['run'] = [run_dir for _ in range(len(run_df))]
                run_dfs.append(run_df)
        else:
            run_df = pd.read_csv(exp_path / variant_dir / 'metrics.csv')
            if exclude is not None:
                run_df = run_df[~run_df['object_name'].isin([exclude])]
            # Add run dir as another column
            run_df['run'] = [variant_dir for _ in range(len(run_df))]
            run_dfs.append(run_df)
            
        variant_df = pd.concat(run_dfs)
        variant_dfs.append(variant_df)
    exp_df = pd.concat(variant_dfs)
    assert np.all(exp_df.isnull().sum() == 0) # assert no NaNs
    return exp_df


def get_unique_evaluation_scenes(exp_df):
    # Get scenes
    run_names = exp_df.run.unique()
    evals = set()
    for run_name in run_names:
        eval_name = run_name.split('-')[0]
        evals.add(eval_name)
    return evals


def get_htmls_mds(exp_df, evals):
    htmls = []
    mds = []
    for eval_name in evals:
        eval_df = exp_df.loc[exp_df['run'].str.contains(eval_name)]
        eval_df = eval_df.drop('scene_idx', axis=1).groupby(['run'])
        count = eval_df.count()['execution']
        sub_max = count != count.max()
        if sub_max.sum() > 0:
            print(f"Warning: {count.loc[sub_max].index[0]} only has {np.array(count.loc[sub_max])[0]} / {count.max()} trials")
        df = eval_df.mean().round(2).astype(str) + " +/- " + eval_df.std().round(1).astype(str)

        htmls.append((eval_name, HTML(df.to_html())))
        mds.append((eval_name, df.to_markdown()))
    return htmls, mds

    
def get_experiment_results(exp_path, multirun=True, exclude=None):
    exp_df = get_experiment_df(exp_path, multirun=multirun, exclude=exclude)
    evals = get_unique_evaluation_scenes(exp_df)
    htmls, mds = get_htmls_mds(exp_df, evals)
    return exp_df, evals, htmls, mds
    

def print_htmls_mds(htmls, mds):
    # Pretty print
    for (eval_name, html) in htmls:
        display(Markdown(f'{eval_name} evaluation'))
        display(html)

## For Qualitative Results

In [ ]:
def sample_gifs(eval_name, variant, exp_df, success=False, num_samples=5, seed=0, obj_type=None, multirun=False):
    if multirun:
        eval_df = exp_df.loc[exp_df['run'].str.contains(eval_name)]
        variant_df = eval_df.loc[eval_df['run'].str.contains(variant)]
    else:
        variant_df = exp_df.loc[exp_df['run'].str.contains(variant)]
    full_variant_name = variant_df['run'].iloc[0]
    selection = variant_df['execution'] == success

    # turn into gifs
    success_df = variant_df.loc[selection]
    if obj_type is not None:
        success_df = success_df.loc[ success_df['object_name'].str.contains(obj_type)] 
    
    paths = success_df[["object_name", "scene_idx"]].apply(lambda x: "_".join([str(y) for y in x]), axis=1)

    glob_pattern = f'*/{full_variant_name}' if multirun else full_variant_name
    variant_path_list = list(exp_path.glob(glob_pattern))
    assert len(variant_path_list) == 1
    variant_path = variant_path_list[0]

    idxs = paths.index
    np.random.seed(seed)
    sampled_idxs = np.random.choice(idxs, size=num_samples, replace=False)
    ims = []
    im_paths = []
    for idx in sampled_idxs:
        path = variant_path / 'gifs' / f'{paths[idx]}.gif'
        print(variant_path / 'avis' / f'{paths[idx]}.avi')
        im = Image(open(path,'rb').read())
        display(im)
        ims.append(im)
        im_paths.append(path)
    return ims, im_paths

# Results 

# Experiment: per object models

In [ ]:
root_dir = Path(os.path.abspath("")) / '..' / '..' / '..' / 'data' / 'pybullet_eval'
exp_name = 'perobj_reach_grasp'
exp_path = Path(f'{root_dir}/{exp_name}')
exp_df, evals, htmls, mds = get_experiment_results(exp_path, multirun=False)
print_htmls_mds(htmls, mds)

In [ ]:
# Get success examples
pd.options.display.max_colwidth = 100
variant = 'GF_learned-perobj_deepsdf_grasp1k_obs0.1_ik_v2partial_2022-11-04_000841'
ims, im_paths = sample_gifs('acronym', variant, exp_df, success=True, num_samples=5, multirun=False)

# Experiment: intra-category models

In [ ]:
root_dir = Path(os.path.abspath("")) / '..' / '..' / '..' / 'data' / 'pybullet_eval'
exp_name = 'multobj_reach_grasp'
exp_path = Path(f'{root_dir}/{exp_name}')
exp_df, evals, htmls, mds = get_experiment_results(exp_path, multirun=False)
print_htmls_mds(htmls, mds)